<a href="https://colab.research.google.com/github/amrosnik/adventure_scrape/blob/master/INFINNERATOR_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

InFINNerator: an RNN for generating Finn the Human speech

Here's my first attempt at creating an RNN. I will be using a homemade dataset of Adventure Time transcripts of Finn dialogue to generate "new" Finn speech! 

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Let's first look at the data. I've outlined elsewhere how I formatted the dataset, but for now let's unpack the pickled Finn data.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import unicodedata
import string
import pickle
import random
import numpy as np
import cloudpickle as cp
from urllib.request import urlopen
import time
import math
import torch.optim as optim

## initial inspiration from from https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html 
torch.manual_seed(1)
all_letters = string.punctuation + string.digits + string.ascii_lowercase + string.whitespace
n_letters = len(all_letters)+1
url = 'https://github.com/amrosnik/adventure_scrape/raw/master/finn_just_dialogue.pkl'
finn_data = cp.load(urlopen(url)) 

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
finn_text = [unicodeToAscii(line) for line in finn_data]
finn_dialogue = [line for line in finn_text if len(line) > 0 ]
random.shuffle(finn_dialogue)

# concatenate all lines into one big corpus:
finn_corpus = '\n'.join(str(elem) for elem in finn_dialogue)
finn_corpus = list(finn_corpus)
finn_corpus = np.asarray(finn_corpus)

## separate dataset into lines of equal number of chars 
line_size = 64
num_lines = round(len(finn_corpus)/line_size)
randomized_finn = [(finn_corpus[i:i+line_size]) for i in range(0, len(finn_corpus), line_size)]

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
# One-hot matrix of first to last letters (not including EOS) for input
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    return torch.LongTensor(letter_indexes)

   # Random item from a list
def randomChoice(l):
      return l[random.randint(0, len(l) - 1)]

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample(minibatch_k=32):
    limits = len(finn_corpus)-(line_size+1)
    idxs = np.asarray(np.random.randint(0,limits,size=minibatch_k))
    inputs = np.array([finn_corpus[idx:idx+line_size] for idx in idxs])
    targets = np.array([finn_corpus[(idx+1):idx+1+line_size] for idx in idxs])
    inputs = inputs.T
    targets = targets.T
    input_line_tensor = torch.zeros(line_size, minibatch_k, n_letters,dtype=torch.float32)  ## size 64,32,101
    for line in range(len(inputs[0])):
       for li, letter in enumerate(inputs[line]):
          #print(i,line,letter)
          input_line_tensor[li][line][letterToIndex(letter)] = 1
    target_line_tensor = torch.zeros(line_size, minibatch_k,dtype=torch.long)
    for line in range(len(targets[0])):
       for li,letter in enumerate(targets[line]):
          target_line_tensor[li][line] = letterToIndex(letter)
    return inputs,targets,input_line_tensor, target_line_tensor 

In [0]:
from torch.autograd import Variable

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, n_layers,batch_first=False)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        input_view = input.view(1,batch_size,-1)
        output, hidden = self.rnn(input_view, hidden)
        output = self.decoder(output[0])
        output = self.softmax(output)
        return output, hidden


    def init_hidden(self, batch_size):
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
               Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))


In [0]:
criterion = nn.NLLLoss()

learning_rate = 0.0001
hidden_size = 512
batch_size = 32
rnn = CharRNN(n_letters, hidden_size,n_letters,n_layers=2)
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

def train(ins,ts,input_line_tensor, target_line_tensor):
    hc = rnn.init_hidden(batch_size)
    optimizer.zero_grad()

    loss = torch.empty(1)
    output = torch.empty((input_line_tensor.size(1),input_line_tensor.size(2)),requires_grad=True)

    seq_len = input_line_tensor.size(0)
    for i in range(seq_len):
        output, hc = rnn(input_line_tensor[i], hc)
        loss += criterion(output, target_line_tensor[i]) / seq_len
    if (loss > 10):
      print(loss,ins)
    loss.backward()
  
    optimizer.step()
    return output, loss.item()



In [0]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
  
n_iters = 5000
print_every = 50
plot_every = 50
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss
    
    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, total_loss / print_every))
        total_loss = 0

In [0]:
total_loss = 0
n_iters = 250
for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss
    
    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, total_loss / print_every))
        total_loss = 0

In [0]:
max_length = 100
# Sample from a category and starting letter
temperature = 1.0

def sample(start_letter='A', temperature=1.5):
    with torch.no_grad():  # no need to track history in sampling
        input = lineToTensor(start_letter)
        hidden = rnn.init_hidden(1)

        output_name = start_letter
        
        for i in range(max_length):
            output, hidden = rnn(input[0], hidden)
            
            exp_output = torch.exp(temperature * output)
            top_i = torch.multinomial(exp_output, 1)[0]
            letter = all_letters[top_i]
            output_name += letter
            input = lineToTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(start_letters='abc'):
    for start_letter in start_letters:
        print(sample(start_letter))
  
batch_size = 32
hidden_size = 128
prime_sample = random.choice(all_letters)
test = sample(prime_sample)
print(test)

}ltgmh  am r t  .eoehol ,oo lo  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
